In [1]:
import fitparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Parse the .FIT file
def parse_fit_file(file_path):
    fitfile = fitparse.FitFile(file_path)
    
    hill_rep_list = list(fitfile.get_messages('lap'))

    hill_rep_start = hill_rep_list[1].get_value('start_time')
    hill_rep_end = hill_rep_list[-1].get_value('start_time')
    
    data = []

    for record in fitfile.get_messages("record"):
        record_data = {}
        for field in record:
            record_data[field.name] = field.value
        data.append(record_data)

    df = pd.DataFrame(data)
    
    # Print a summary of the columns to understand structure
    print("Columns in .FIT file:", df.columns.tolist())
    
    # Clean and filter necessary columns
    required_columns = ["timestamp", "enhanced_altitude", "distance", "heart_rate"]  # Modify if needed
    for col in required_columns:
        if col not in df.columns:
            print(f"Column '{col}' is missing in the data!")
            return None  # Early exit if required data is unavailable

    # Ensure relevant columns are present and processable
    df = df[required_columns].dropna()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["alt_diff"] = df["enhanced_altitude"].diff()
    df["dist_diff"] = df["distance"].diff()
    df = df.fillna(0)  # Replace NaN values with
    
    df = df[(df["timestamp"] >= hill_rep_start) & (df["timestamp"] <= hill_rep_end)]
    return df

parse_fit_file("./fitfiles/17727285417_ACTIVITY.fit")

Columns in .FIT file: ['activity_type', 'cadence', 'distance', 'enhanced_altitude', 'enhanced_speed', 'fractional_cadence', 'heart_rate', 'position_lat', 'position_long', 'stance_time', 'stance_time_balance', 'stance_time_percent', 'step_length', 'temperature', 'timestamp', 'vertical_oscillation', 'vertical_ratio', 'unknown_107', 'unknown_134', 'unknown_135', 'unknown_136', 'unknown_137', 'unknown_138', 'unknown_140', 'unknown_143', 'unknown_144', 'unknown_87', 'unknown_108', 'unknown_90']


,timestamp,enhanced_altitude,distance,heart_rate,alt_diff,dist_diff
693,2024-12-10 05:31:19,54.2,2262.84,83,0.0,0.68
694,2024-12-10 05:31:20,54.2,2263.46,82,0.0,0.62
695,2024-12-10 05:31:21,54.2,2265.73,81,0.0,2.27
696,2024-12-10 05:31:22,54.2,2267.53,83,0.0,1.80
697,2024-12-10 05:31:23,54.4,2269.10,85,0.2,1.57
...,...,...,...,...,...,...
5237,2024-12-10 06:47:03,52.8,8523.71,139,-0.4,0.83
5238,2024-12-10 06:47:04,52.2,8525.22,138,-0.6,1.51
5239,2024-12-10 06:47:05,51.4,8526.56,137,-0.8,1.34
5240,2024-12-10 06:47:06,50.6,8528.12,136,-0.8,1.56


In [8]:
def detect_repeats(df, elevation_gain_threshold=10, downhill_gain_threshold=10):
    df["lap_number_up"] = 0
    df["lap_number_down"] = 0
    
    # Detect new uphill sections
    df['new_uphill'] = ((df['alt_diff'] > 0) & 
                        (df['alt_diff'].shift(fill_value=0)
                         .rolling(window=10, min_periods=10)
                         .apply(lambda x: (x <= 0).sum(), raw=True) >= 9))
    
    df['new_uphill'] = (df['new_uphill'] == True) & (df['new_uphill'].shift(-1) == False)
    
    df['lap_number_up'] = df['new_uphill'].cumsum()
    
    # Detect new downhill sections
    df['new_downhill'] = ((df['alt_diff'] < 0) & 
                          (df['alt_diff'].shift(fill_value=0)
                           .rolling(window=10, min_periods=10)
                           .apply(lambda x: (x >= 0).sum(), raw=True) >= 9))
    
    df['new_downhill'] = (df['new_downhill'] == True) & (df['new_downhill'].shift(-1) == False)
    
    df['lap_number_down'] = df['new_downhill'].cumsum()
    
    df = df.groupby('lap_number_up').filter(lambda x: x['enhanced_altitude'].max() - x['enhanced_altitude'].min() > elevation_gain_threshold)
    
    
    return df


In [11]:



  
# File path
fit_file_path = "./fitfiles/17727285417_ACTIVITY.fit"  # Replace with actual file path
# fit_file_path = "./fitfiles/17684106977_ACTIVITY.fit"

# Parse the FIT file
activity_data = parse_fit_file(fit_file_path)

if activity_data is not None:
    # Detect repeats
    repeats = detect_repeats(activity_data)
    display(repeats)
    print("Detected Hill Repeats:")
    # display(repeats.groupby('lap_number_up').agg({'timestamp': ['min', 'max'], 'enhanced_altitude': ['min', 'max'], 'dist_diff': 'sum', 'heart_rate': 'mean', 'alt_diff': 'sum'}).reset_index())	
    display(repeats.groupby(['lap_number_up','lap_number_down']).agg({'timestamp': ['min', 'max'], 'enhanced_altitude': ['min', 'max'], 'dist_diff': 'sum', 'heart_rate': 'mean', 'alt_diff': 'sum'}).reset_index())	


        

Columns in .FIT file: ['activity_type', 'cadence', 'distance', 'enhanced_altitude', 'enhanced_speed', 'fractional_cadence', 'heart_rate', 'position_lat', 'position_long', 'stance_time', 'stance_time_balance', 'stance_time_percent', 'step_length', 'temperature', 'timestamp', 'vertical_oscillation', 'vertical_ratio', 'unknown_107', 'unknown_134', 'unknown_135', 'unknown_136', 'unknown_137', 'unknown_138', 'unknown_140', 'unknown_143', 'unknown_144', 'unknown_87', 'unknown_108', 'unknown_90']


,timestamp,enhanced_altitude,distance,heart_rate,alt_diff,dist_diff,lap_number_up,lap_number_down,new_uphill,new_downhill
693,2024-12-10 05:31:19,54.2,2262.84,83,0.0,0.68,0,0,False,False
694,2024-12-10 05:31:20,54.2,2263.46,82,0.0,0.62,0,0,False,False
695,2024-12-10 05:31:21,54.2,2265.73,81,0.0,2.27,0,0,False,False
696,2024-12-10 05:31:22,54.2,2267.53,83,0.0,1.80,0,0,False,False
697,2024-12-10 05:31:23,54.4,2269.10,85,0.2,1.57,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...
5237,2024-12-10 06:47:03,52.8,8523.71,139,-0.4,0.83,49,51,False,False
5238,2024-12-10 06:47:04,52.2,8525.22,138,-0.6,1.51,49,51,False,False
5239,2024-12-10 06:47:05,51.4,8526.56,137,-0.8,1.34,49,51,False,False
5240,2024-12-10 06:47:06,50.6,8528.12,136,-0.8,1.56,49,51,False,False


Detected Hill Repeats:


lap_number_up lap_number_down           timestamp                      \
                                                  min                 max   
0               0               0 2024-12-10 05:31:19 2024-12-10 05:32:20   
1               0               1 2024-12-10 05:32:21 2024-12-10 05:32:58   
2               1               1 2024-12-10 05:32:59 2024-12-10 05:33:51   
3               1               2 2024-12-10 05:33:52 2024-12-10 05:34:31   
4               2               2 2024-12-10 05:34:32 2024-12-10 05:35:25   
..            ...             ...                 ...                 ...   
96             47              49 2024-12-10 06:43:36 2024-12-10 06:44:14   
97             48              49 2024-12-10 06:44:15 2024-12-10 06:45:07   
98             48              50 2024-12-10 06:45:08 2024-12-10 06:45:44   
99             49              50 2024-12-10 06:45:45 2024-12-10 06:46:35   
100            49              51 2024-12-10 06:46:36 2024-12-10 06:47:07   

    enhanced_altitude       dist_diff  heart_rate alt_diff  
                  min   max       sum        mean      sum  
0                54.2  76.6     73.87  120.370968     21.8  
1                55.0  75.4     61.65  126.736842    -20.4  
2                56.4  77.4     68.20  134.320755     21.2  
3                55.4  76.2     62.34  129.350000    -21.0  
4                56.2  77.2     64.32  131.240741     20.8  
..                ...   ...       ...         ...      ...  
96               48.8  70.0     63.42  142.846154    -21.4  
97               49.2  70.2     66.76  140.811321     20.8  
98               49.0  69.2     56.62  141.135135    -20.6  
99               50.0  70.2     63.75  141.980392     20.8  
100              50.0  69.8     57.07  145.312500    -20.0  

[101 rows x 9 columns]

In [90]:
# Perform groupby and aggregation
result = (
    repeats.groupby('lap_number')
    .agg({
        'enhanced_altitude': ['min', 'max'],
        'dist_diff': 'sum',
        'timestamp': ['min', 'max']
    })
    .reset_index()  # Reset index to make `lap_number` a column
)

# Filter groups where `alt_diff` > 10 using multi-level access
result = result[result[('enhanced_altitude', 'max')] - result[('enhanced_altitude', 'min')] > 10]
result.reset_index(drop=True, inplace=True)
result['alt_diff'] = result[('enhanced_altitude', 'max')] - result[('enhanced_altitude', 'min')]

# Sort by `alt_diff` in descending order
# result = result.sort_values(by=('alt_diff', 'sum'), ascending=False)

display(result)

lap_number enhanced_altitude       dist_diff           timestamp  \
                            min   max       sum                 min   
0           0              54.2  76.6    135.52 2024-12-10 05:31:19   
1           1              55.4  77.4    130.54 2024-12-10 05:32:59   
2           2              55.4  77.2    125.88 2024-12-10 05:34:32   
3           3              55.8  77.2    121.53 2024-12-10 05:36:05   
4           4              56.0  78.0    126.18 2024-12-10 05:37:36   
5           5              55.8  77.6    126.45 2024-12-10 05:39:08   
6           6              56.2  77.6    124.01 2024-12-10 05:40:37   
7           7              55.4  77.2    122.83 2024-12-10 05:42:10   
8           8              55.8  77.2    133.39 2024-12-10 05:43:40   
9           9              55.2  77.8    122.59 2024-12-10 05:45:12   
10         10              55.0  77.0    129.75 2024-12-10 05:46:42   
11         11              54.6  76.2    123.14 2024-12-10 05:48:14   
12         12              54.4  75.6    124.94 2024-12-10 05:49:45   
13         13              53.0  75.8    126.17 2024-12-10 05:51:17   
14         14              52.8  74.0    125.58 2024-12-10 05:52:48   
15         15              51.6  73.6    119.54 2024-12-10 05:54:19   
16         16              51.2  73.2    129.89 2024-12-10 05:55:50   
17         17              50.8  72.6    115.91 2024-12-10 05:57:22   
18         18              50.8  72.6    131.00 2024-12-10 05:58:53   
19         19              49.6  71.8    124.45 2024-12-10 06:00:25   
20         20              50.0  71.4    126.84 2024-12-10 06:01:55   
21         21              48.8  71.4    120.45 2024-12-10 06:03:29   
22         22              48.6  70.4    123.45 2024-12-10 06:05:00   
23         23              48.8  70.2    121.57 2024-12-10 06:06:30   
24         24              49.2  70.4    123.80 2024-12-10 06:08:02   
25         25              48.2  69.8    119.92 2024-12-10 06:09:34   
26         26              47.6  69.4    124.24 2024-12-10 06:11:04   
27         27              48.4  69.2    123.44 2024-12-10 06:12:35   
28         28              48.2  70.0    126.69 2024-12-10 06:14:06   
29         29              47.8  69.8    123.63 2024-12-10 06:15:37   
30         30              48.4  69.8    121.51 2024-12-10 06:17:08   
31         31              47.8  69.8    122.78 2024-12-10 06:18:38   
32         32              48.4  70.2    128.77 2024-12-10 06:20:09   
33         33              48.0  69.6    121.45 2024-12-10 06:21:40   
34         34              48.6  70.0    118.26 2024-12-10 06:23:10   
35         35              48.6  70.4    132.42 2024-12-10 06:24:40   
36         36              49.2  70.6    129.72 2024-12-10 06:26:11   
37         37              49.0  70.8    126.84 2024-12-10 06:27:42   
38         38              49.2  71.0    131.53 2024-12-10 06:29:12   
39         39              49.0  71.2    126.22 2024-12-10 06:30:43   
40         40              49.4  71.0    128.33 2024-12-10 06:32:13   
41         41              49.2  71.0    126.60 2024-12-10 06:33:44   
42         42              49.0  70.4    125.52 2024-12-10 06:35:14   
43         43              49.0  70.2    124.79 2024-12-10 06:36:45   
44         44              49.0  70.2    129.77 2024-12-10 06:38:16   
45         45              49.2  70.6    124.42 2024-12-10 06:39:47   
46         46              49.0  70.8    128.04 2024-12-10 06:41:16   
47         47              48.8  70.6    123.25 2024-12-10 06:42:47   
48         48              49.0  70.2    123.38 2024-12-10 06:44:15   
49         49              50.0  70.2    120.82 2024-12-10 06:45:45   

                       alt_diff  
                   max           
0  2024-12-10 05:32:58     22.4  
1  2024-12-10 05:34:31     22.0  
2  2024-12-10 05:36:04     21.8  
3  2024-12-10 05:37:35     21.4  
4  2024-12-10 05:39:07     22.0  
5  2024-12-10 05:40:36     21.8  
6  2024-12-10 05:42:09     21.4  
7  2